In [0]:
from pyspark.sql.functions import when,sum

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, date_format, col,count

# Create a SparkSession
spark = SparkSession.builder.appName("join").getOrCreate()

# Sample Storagehouse data
data1=[("Room1",1,1),
       ("Room1",2,10),
       ("Room1",3,5),
       ("Room2",1,2),
       ("Room2",2,2),
       ("Room3",4,1)]
df_st=spark.createDataFrame(data1,["name","product_id","units"])

data2=[
        (1,"Mencollection",5,50,40),
(2,"Girlscollection",5,5,5),
(3,"Childcollection",2,10,10),
(4,"Womencollection",4,10,20)
]

df_pr=spark.createDataFrame(data2,["product_id","product_name","Width","Length","Height"])

df_st.show()

df_pr.show()


+-----+----------+-----+
| name|product_id|units|
+-----+----------+-----+
|Room1|         1|    1|
|Room1|         2|   10|
|Room1|         3|    5|
|Room2|         1|    2|
|Room2|         2|    2|
|Room3|         4|    1|
+-----+----------+-----+

+----------+---------------+-----+------+------+
|product_id|   product_name|Width|Length|Height|
+----------+---------------+-----+------+------+
|         1|  Mencollection|    5|    50|    40|
|         2|Girlscollection|    5|     5|     5|
|         3|Childcollection|    2|    10|    10|
|         4|Womencollection|    4|    10|    20|
+----------+---------------+-----+------+------+



<h3>warehouse_name | volume </h3>
<h3>volume = length * width * height</h3>

In [0]:
df_st.join(df_pr,df_st.product_id==df_pr.product_id,"inner").show()

# another way of join
result_df = df_st.join(df_pr, on="product_id", how="inner")
result_df.show()


+-----+----------+-----+----------+---------------+-----+------+------+
| name|product_id|units|product_id|   product_name|Width|Length|Height|
+-----+----------+-----+----------+---------------+-----+------+------+
|Room1|         1|    1|         1|  Mencollection|    5|    50|    40|
|Room2|         1|    2|         1|  Mencollection|    5|    50|    40|
|Room1|         2|   10|         2|Girlscollection|    5|     5|     5|
|Room2|         2|    2|         2|Girlscollection|    5|     5|     5|
|Room1|         3|    5|         3|Childcollection|    2|    10|    10|
|Room3|         4|    1|         4|Womencollection|    4|    10|    20|
+-----+----------+-----+----------+---------------+-----+------+------+

+----------+-----+-----+---------------+-----+------+------+
|product_id| name|units|   product_name|Width|Length|Height|
+----------+-----+-----+---------------+-----+------+------+
|         1|Room1|    1|  Mencollection|    5|    50|    40|
|         1|Room2|    2|  Mencollec

In [0]:
result_df.withColumn("volume",result_df.Width*result_df.Length*result_df.Height)\
    .groupBy(col("name").alias("warehouse_name")).agg(sum("volume").alias("volume")).show()

+--------------+------+
|warehouse_name|volume|
+--------------+------+
|         Room3|   800|
|         Room2| 10125|
|         Room1| 10325|
+--------------+------+

